<a href="https://colab.research.google.com/github/IkeLyons/HW4-Sequence-Models/blob/main/SequenceModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework I am going to try to generate text based off of the script of Star Wars (dataset link: https://www.kaggle.com/xvivancos/star-wars-movie-scripts). DESCRIBE HERE WHY THIS NEEDS TO BE SOLVED USING SEQUENCE MODELS!!!!!!!!!!!!!!!!! 

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download xvivancos/star-wars-movie-scripts

  0% 0.00/158k [00:00<?, ?B/s]
100% 158k/158k [00:00<00:00, 49.8MB/s]


In [6]:
! unzip star-wars-movie-scripts.zip

Archive:  star-wars-movie-scripts.zip
  inflating: SW_EpisodeIV.txt        
  inflating: SW_EpisodeV.txt         
  inflating: SW_EpisodeVI.txt        
  inflating: wordcloud_masks/r2d2.png  
  inflating: wordcloud_masks/rebel alliance.png  
  inflating: wordcloud_masks/vader.png  
  inflating: wordcloud_masks/yoda.png  


We are using TensorFlow and some of the Keras utilties in order to solve this problem.

In [7]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np

In [14]:
text = open('SW_EpisodeIV.txt', 'rb').read().decode(encoding='utf-8')
print(text[:500])

"character" "dialogue"
"1" "THREEPIO" "Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness!"
"2" "THREEPIO" "We're doomed!"
"3" "THREEPIO" "There'll be no escape for the Princess this time."
"4" "THREEPIO" "What's that?"
"5" "THREEPIO" "I should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister..."
"6" "LUKE" "Hurry up!  Come with me!  What are you waiting for?!  Get in gear!"
"7" "THREEPIO" "Artoo! Art


We can see that the script has 3 main parts: the line number, the character speaking, and what the character said. It will be interesting to see how the models are able to recreate this.

In [16]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Task 1

The first thing that the model needs is a way to translate from the characters to ids that represent them.

In [17]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(chars), mask_token=None)

We also need the reverse of the above layer in order to get the human readable text again.

In [37]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [38]:
converted_text = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
converted_text

<tf.Tensor: shape=(78278,), dtype=int64, numpy=array([ 4, 49, 54, ...,  8,  4,  1])>

In [46]:
dataset = tf.data.Dataset.from_tensor_slices(converted_text)

We are going to need a function that splits a string into what the model sees, and the target text for the model to genenrate.

In [40]:
def xysplit(text):
    X = text[:-1]
    y = text[1:]
    return X, y

print(xysplit(list("test_text")))

(['t', 'e', 's', 't', '_', 't', 'e', 'x'], ['e', 's', 't', '_', 't', 'e', 'x', 't'])


In [47]:
seq_length = 50
seqs = dataset.batch(seq_length+1, drop_remainder=True)

dataset = seqs.map(xysplit)

Input : [b'"' b'c' b'h' b'a' b'r' b'a' b'c' b't' b'e' b'r' b'"' b' ' b'"' b'd'
 b'i' b'a' b'l' b'o' b'g' b'u' b'e' b'"' b'\n' b'"' b'1' b'"' b' ' b'"'
 b'T' b'H' b'R' b'E' b'E' b'P' b'I' b'O' b'"' b' ' b'"' b'D' b'i' b'd'
 b' ' b'y' b'o' b'u' b' ' b'h' b'e' b'a']
Target: [b'c' b'h' b'a' b'r' b'a' b'c' b't' b'e' b'r' b'"' b' ' b'"' b'd' b'i'
 b'a' b'l' b'o' b'g' b'u' b'e' b'"' b'\n' b'"' b'1' b'"' b' ' b'"' b'T'
 b'H' b'R' b'E' b'E' b'P' b'I' b'O' b'"' b' ' b'"' b'D' b'i' b'd' b' '
 b'y' b'o' b'u' b' ' b'h' b'e' b'a' b'r']


The next step is to split the dataset into batches

In [48]:
buffer_size = 10000
batch_size = 50

dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


Now that our dataset is ready, it is time to create the actual model